### Structured Streaming


#### Exemplo de lectura de ficheiros - Window - Exercicio I


https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#overview

In [ ]:
# Xestionamos unha cadea de cafeterías e temos un repositorio común onde cada tenda
# sube un ficheiro coa información das súas vendas cada poucos minutos
# Os datos de cada venda son: data/hora (timestamp), tenda, tipo (bebida,comida,...), produto
# Queremos que o noso sistema mostre cantos produtos está vendendo cada tenda nun intervalo dunha hora
# lanzando a consulta cada media hora

In [ ]:
# No caso de de lanzar a aplicación desde consola debemos configurar a session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("StreamingFilesWindowed") \
    .getOrCreate() 

In [ ]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import *

userSchema = StructType().add("hora", "timestamp").add("tenda","string").add("tipo","string").add("produto", "string")

orixe = spark \
    .readStream \
    .option("sep", ";") \
    .schema(userSchema) \
    .csv("file:///home/hduser/input/csv")

#procesado = orixe.groupBy( window(orixe.hora, "60 minutes", "30 minutes"), orixe.tenda, orixe.produto).count()
procesado = orixe.groupBy( window(orixe.hora, "60 minutes", "30 minutes"), orixe.tenda).count()

consulta = procesado \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate",False) \
    .start()

consulta.awaitTermination()